# Hello

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import sentencepiece as spm

# Import dataset từ TFDS

In [ ]:
DATASET_NAME = "wmt14_translate/fr-en"

train_ds = tfds.load(DATASET_NAME, split="train", shuffle_files=True)
valid_ds = tfds.load(DATASET_NAME, split="validation")


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

...

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

# Subset - Dataset full ~40M câu → không train nổi

In [ ]:
train_ds = tfds.load(
    DATASET_NAME,
    split="train[:2%]"   # ~800k câu
)

valid_ds = tfds.load(
    DATASET_NAME,
    split="validation"
)

# Train tokenizer (SentencePiece, joint vocab)

In [ ]:
def extract_text(ds, lang, max_lines=200_000):
    texts = []
    for i, ex in enumerate(ds):
        if i >= max_lines:
            break
        texts.append(ex["translation"][lang].numpy().decode("utf-8"))
    return texts

fr_text = extract_text(train_ds, "fr")
en_text = extract_text(train_ds, "en")

In [ ]:
with open("spm_input.txt", "w", encoding="utf-8") as f:
    for line in fr_text + en_text:
        f.write(line + "\n")

spm.SentencePieceTrainer.train(
    input="spm_input.txt",
    model_prefix="spm",
    vocab_size=16000,
    model_type="bpe",
    pad_id=0,
    unk_id=1,
    bos_id=2,
    eos_id=3
)


In [ ]:
sp = spm.SentencePieceProcessor()
sp.load("spm.model")


# Encode câu (RẤT QUAN TRỌNG)

In [ ]:
def encode(src, tgt):
    src_ids = sp.encode(src.numpy().decode("utf-8"))
    tgt_ids = sp.encode(tgt.numpy().decode("utf-8"))

    # Decoder input và output
    tgt_in  = [2] + tgt_ids        # <bos> + y
    tgt_out = tgt_ids + [3]        # y + <eos>

    return src_ids, tgt_in, tgt_out


In [ ]:
def tf_encode(example):
    src, tgt_in, tgt_out = tf.py_function(
        func=encode,
        inp=[
            example["translation"]["fr"],
            example["translation"]["en"]
        ],
        Tout=[tf.int32, tf.int32, tf.int32]
    )
    src.set_shape([None])
    tgt_in.set_shape([None])
    tgt_out.set_shape([None])
    return src, tgt_in, tgt_out


# Tạo Dataset Cho Training

In [ ]:
BATCH_SIZE = 64
MAX_LEN = 100

def filter_length(src, tgt_in, tgt_out):
    return tf.logical_and(
        tf.size(src) <= MAX_LEN,
        tf.size(tgt_in) <= MAX_LEN
    )

train_ds = train_ds.map(tf_encode, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.filter(filter_length)

train_ds = train_ds.padded_batch(
    BATCH_SIZE,
    padded_shapes=([None], [None], [None]),
    padding_values=(0, 0, 0)
).prefetch(tf.data.AUTOTUNE)
